In [78]:
import os
from PIL import Image
import time
import json
import yaml

In [66]:
# Image Resize (width: 640): 너무 작다.

def resize_image_to_width_640(filename):
    with Image.open(filename) as img:
        width, height = img.size
        ratio = height / width
        
        new_width = 640
        new_height = int(ratio * new_width)

        # Image.Dither.None 옵션을 사용하여 속도 약 2배 빨라짐
        im_resize = img.resize((new_width, new_height), Image.Dither.NONE)

    os.remove(filename)
    im_resize.save(filename)

In [69]:
# Image Resize (height: 640)

def resize_image_to_height_640(filename):
    with Image.open(filename) as img:
        width, height = img.size
        ratio = width / height
        
        new_height = 640
        new_width = int(ratio * new_height)

        # Image.Dither.None 옵션을 사용하여 속도 약 2배 빨라짐
        im_resize = img.resize((new_width, new_height), Image.Dither.NONE)

    os.remove(filename)
    im_resize.save(filename)

In [70]:
# Images Resize by recursive

def resize_images_recursive(src):
    filenames = os.listdir(src)

    for filename in filenames:
        if os.path.isdir(os.path.join(src, filename)):
            resize_images_recursive(os.path.join(src, filename))
            
        if os.path.isfile(os.path.join(src, filename)):
            if filename != '.DS_Store':
                if os.path.splitext(filename)[1] == '.jpg':
                    resize_image_to_height_640(os.path.join(src, filename))

In [72]:
# Resize Train Data

start = time.time()
resize_images_recursive('./Data/train/원천데이터')
print("time :", time.time() - start)

time : 7316.926134109497


In [71]:
# Resize Validation Data

start = time.time()
resize_images_recursive('./Data/validation/원천데이터')
print("time :", time.time() - start)

time : 910.5802710056305


In [75]:
# Get Classes List

model_classes = []

def get_model_classes(src):
    with open(src, 'r') as jess:
        jess_dict = json.load(jess)
        
    rawDataID = jess_dict['rawDataInfo']['rawDataID'][9:21]
    if rawDataID not in model_classes:
        model_classes.append(rawDataID)

In [76]:
# Get Classes List by recursive

def get_classes_recursive(src):
    filenames = os.listdir(src)

    for filename in filenames:
        if os.path.isdir(os.path.join(src, filename)):
            get_classes_recursive(os.path.join(src, filename))
            
        if os.path.isfile(os.path.join(src, filename)):
            if filename != '.DS_Store':
                if os.path.splitext(filename)[1] == '.json':
                    get_model_classes(os.path.join(src, filename))

In [79]:
# Get Classes

start = time.time()
get_classes_recursive('./Data/train/라벨링데이터')
print("time :", time.time() - start)

time : 75.73377180099487


In [81]:
model_classes[0]

'KI_056_17_WH'

In [82]:
sorted_model_classes = sorted(model_classes)
sorted_model_classes[0]

'HY_025_17_GR'

In [96]:
sorted_model_classes

['HY_025_17_GR',
 'HY_025_17_RE',
 'HY_025_17_WH',
 'HY_025_18_BK',
 'HY_025_18_BL',
 'HY_025_18_GR',
 'HY_025_18_WH',
 'HY_025_19_RE',
 'HY_025_19_WH',
 'HY_041_17_BK',
 'HY_041_17_BL',
 'HY_041_17_GR',
 'HY_041_17_WH',
 'HY_041_18_BK',
 'HY_041_18_BL',
 'HY_041_18_GR',
 'HY_041_18_WH',
 'HY_041_19_BK',
 'HY_041_19_BL',
 'HY_041_19_GR',
 'HY_041_19_WH',
 'HY_041_20_BK',
 'HY_041_20_GR',
 'HY_041_20_WH',
 'HY_041_21_BK',
 'HY_041_21_GR',
 'HY_041_21_WH',
 'HY_042_20_BK',
 'HY_042_20_GR',
 'HY_050_17_BL',
 'HY_050_17_GR',
 'HY_050_17_RE',
 'HY_050_18_BK',
 'HY_050_18_BL',
 'HY_050_18_GR',
 'HY_050_18_WH',
 'HY_053_19_BL',
 'HY_053_19_GR',
 'HY_053_20_BL',
 'HY_053_20_GR',
 'HY_053_20_RE',
 'HY_053_20_WH',
 'HY_053_21_BL',
 'HY_053_21_RE',
 'HY_053_21_WH',
 'HY_054_17_GR',
 'HY_054_18_BK',
 'HY_054_18_BL',
 'HY_054_18_RE',
 'HY_054_19_BK',
 'HY_054_19_BL',
 'HY_054_19_GR',
 'HY_054_19_WH',
 'HY_054_20_BK',
 'HY_054_20_BL',
 'HY_054_20_GR',
 'HY_054_21_BK',
 'HY_058_17_BK',
 'HY_058_17_GR

In [83]:
all_parts = { "P00":"차량전체", "P01":"프론트범퍼", "P02":"리어범퍼", "P03":"타이어(휠)", "P04":"A필러",
       "P05":"C필러", "P06":"사이드미러", "P07":"앞도어", "P08":"뒷도어", "P09":"라디에이터그릴",
       "P10":"헤드램프", "P11":"리어램프", "P12":"보닛", "P13":"트렁크", "P14":"루프", }

In [90]:
# ignore_parts = ["P03", "P04", "P05", "P06", "P07", "P08", "P12", "P13", "P14", ]

In [92]:
# Make Label with parts

def make_label_with_parts(src):
    with open(src, 'r') as jess:
        jess_dict = json.load(jess)
    
    rawDataID = jess_dict['rawDataInfo']['rawDataID'][9:21]
    resolution = jess_dict['rawDataInfo']['resolution']
    width, height = int(resolution.split('*')[0]), int(resolution.split('*')[1])
    
    with open(os.path.splitext(src)[0]+'.txt', 'w') as f:
        for obj in jess_dict['learningDataInfo']['objects']:
            classID = obj['classId'].split('.')[0]
            cx = (int(obj['coords']['tl']['x']) + int(obj['coords']['tr']['x'])) / 2 / width
            cy = (int(obj['coords']['tl']['y']) + int(obj['coords']['bl']['y'])) / 2 / height
            w = (int(obj['coords']['tr']['x']) - int(obj['coords']['tl']['x'])) / width
            h = (int(obj['coords']['bl']['y']) - int(obj['coords']['tl']['y'])) / height

            txt = str(sorted_model_classes.index(rawDataID)) + ' ' + str(cx) + ' ' + str(cy) + ' ' + str(w) + ' ' + str(h) + '\n'

            f.write(txt)    
    
    os.remove(src)

In [93]:
# Make Labels with parts by recursive

def make_labels_with_parts_recursive(src):
    filenames = os.listdir(src)

    for filename in filenames:
        if os.path.isdir(os.path.join(src, filename)):
            make_labels_with_parts_recursive(os.path.join(src, filename))
            
        if os.path.isfile(os.path.join(src, filename)):
            if filename != '.DS_Store':
                if os.path.splitext(filename)[1] == '.json':
                    make_label_with_parts(os.path.join(src, filename))

In [95]:
# Make Labels for train

start = time.time()
make_labels_with_parts_recursive('./Data/train/라벨링데이터')
print("time :", time.time() - start)

time : 200.46526217460632


In [94]:
# Make Labels for validation

start = time.time()
make_labels_with_parts_recursive('./Data/validation/라벨링데이터')
print("time :", time.time() - start)

time : 21.17518401145935


In [100]:
# Make dict from sorted_model_classes

dict_classes = {i : c for i, c in enumerate(sorted_model_classes)}
print(dict_classes)

{0: 'HY_025_17_GR', 1: 'HY_025_17_RE', 2: 'HY_025_17_WH', 3: 'HY_025_18_BK', 4: 'HY_025_18_BL', 5: 'HY_025_18_GR', 6: 'HY_025_18_WH', 7: 'HY_025_19_RE', 8: 'HY_025_19_WH', 9: 'HY_041_17_BK', 10: 'HY_041_17_BL', 11: 'HY_041_17_GR', 12: 'HY_041_17_WH', 13: 'HY_041_18_BK', 14: 'HY_041_18_BL', 15: 'HY_041_18_GR', 16: 'HY_041_18_WH', 17: 'HY_041_19_BK', 18: 'HY_041_19_BL', 19: 'HY_041_19_GR', 20: 'HY_041_19_WH', 21: 'HY_041_20_BK', 22: 'HY_041_20_GR', 23: 'HY_041_20_WH', 24: 'HY_041_21_BK', 25: 'HY_041_21_GR', 26: 'HY_041_21_WH', 27: 'HY_042_20_BK', 28: 'HY_042_20_GR', 29: 'HY_050_17_BL', 30: 'HY_050_17_GR', 31: 'HY_050_17_RE', 32: 'HY_050_18_BK', 33: 'HY_050_18_BL', 34: 'HY_050_18_GR', 35: 'HY_050_18_WH', 36: 'HY_053_19_BL', 37: 'HY_053_19_GR', 38: 'HY_053_20_BL', 39: 'HY_053_20_GR', 40: 'HY_053_20_RE', 41: 'HY_053_20_WH', 42: 'HY_053_21_BL', 43: 'HY_053_21_RE', 44: 'HY_053_21_WH', 45: 'HY_054_17_GR', 46: 'HY_054_18_BK', 47: 'HY_054_18_BL', 48: 'HY_054_18_RE', 49: 'HY_054_19_BK', 50: 'HY_0

In [101]:
# Make yaml file for YOLOv5

yaml_file = 'car_model.yaml'

yaml_data = dict(
    path = "data",
    train = "train",
    val = "validation",
    nc = len(dict_classes),
    names = dict_classes
)

with open(yaml_file, 'w') as f:
    yaml.dump(yaml_data, f, explicit_start = True, default_flow_style = False)